In [2]:
%matplotlib inline
import numpy as np
from figures import er_replay_capacity as fig_1

### Replay capacity analysis

How many different sequences can a network reliably replay? Mathematically, this is equivalent to asking how many sequences can be specified purely through an unordered *set* of involved ensembles (marked by lingering hyperexcitability), and through the first ensemble in the sequence. This so-called replay capacity will of course depend on the network architecture. In this analysis, we restrict our analysis to networks with binary, directed connections.

Importantly, there should be a nontrivial connectivity matrix that maximizes the replay capacity of a network. If the network has no connections, no sequences can be specified by a set of hyperexcitable ensembles: after the first ensemble activates, all other hyperexcitable ensembles will activate with equal probability, since none of them receives input from the first ensemble. On the other hand, no sequences can be specified in a fully connected network either, since after the first ensemble activates, again all other hyperexcitable ensembles will activate with equal probability, since they always receive increased but equal input from the first ensemble.

Here we quantify the replay capacity of networks with a variety of different connectivities. To perform this calculation we precisely define the replay capacity of a network to be the number of replayable sequences it contains, where we call a sequence of ensembles replayable if the following is true:

Given the first ensemble, only one possible path through the *set* of the later ensembles can be traced by following the connections present in the network.

This simply quantifies the notion that if one knows the *set* of ensembles in the sequence as well as the starting ensemble, then there should be no ambiguity in determining the order. For example:

<img src="files/images/reliable_replayability_example.png" />

We define the *replay capacity* $R_L(C)$ of a network given its connectivity structure $C$ (a binarized version of its weight matrix) as the number of reliably replayable sequences of length $L$ that exist in the network. Specifically:

$$R_L(C) = \sum_\limits{path \in \{ \textrm{paths of length } L \} } \mathbb{1}[path \textrm{ is replayable }].$$


Notably $R_L(C) = 0$ when $C$ is completely unconnected (because there are no paths to sum over) and when $C$ is fully connected (because while all possible paths exist, none is replayable).

For simplicity, we'll only consider paths that start and end at different nodes ($i \neq j$), which will be a lower limit on the total number of replayable paths. For a given random network, one can calculate $E[R_L(C)]$ in the following way:

$$E[R_L(C)] = E\left[\sum_{i, j} n_R^L(i, j) \right] = 
\sum_{i, j} E\left[n_R^L(i, j)\right]$$

where $n_R^L(i, j)$ is the number of reliably replayable paths that start at $i$ and end at $j$. However, if the nodes are indexed randomly, then $E[n_R^L(i, j)] = E[n_R^L(k, l)]$, and so:

$$ \sum_{i, j} E\left[n_R^L(i, j)\right] = N(N-1) E\left[n_R^L(i, j)\right]$$
Thus, one only needs to calculate the expected number of paths between two randomly chosen nodes.

In certain cases the following further simplification is also useful:

$$E\left[n_R^L(i, j)\right] = E\left[ \sum_{\textrm{possible paths from i to j}} \mathbb{1} \left[\textrm{path exists and is replayable} \right] \right]$$

$$ = \sum_{\textrm{possible paths from i to j}} E\left[ \mathbb{1} \left[\textrm{path exists and is replayable} \right] \right]$$

$$ = \sum_{\textrm{possible paths from i to j}} p\left(\textrm{path exists and is replayable} \right).$$

Again, if nodes are indexed randomly in the graph generation process, then we have:

$$ \sum_{\textrm{possible paths from i to j}} p\left(\textrm{path exists and is replayable} \right)$$

$$ = (N-2)(N-3)...(N-L+1) p\left(\textrm{path exists and is replayable} \right)$$

$$ = \cfrac{(N-2)!}{(N-L)!} p\left(\textrm{path exists and is replayable} \right)$$

Therefore the total expected number of replayable paths in a random network is

$$E\left[R_L(C)\right] = \cfrac{N!}{(N-L)!} p\left(\textrm{path exists and is replayable}\right)$$

Thus, the calculation of $E[R_L(C)]$ for a random graph is reduced to the calculation of the probability that there is a replayable path through a single randomly chosen sequence of nodes. In general, this can be decomposed as:

$$p(\textrm{path exists and is replayable})$$

$$ = p(\textrm{path exists}) p(\textrm{path is replayable } | \textrm{ path exists}).$$

And this is simply the probability that, excepting the last node of the sequence, each node connects to the next node in the sequence but to no other nodes in the sequence. As our reference point we will use the maximum number of possible length-$L$ sequences: $N^L$. 

### Erdos-Renyi random graph

In an Erdos-Renyi network, each ordered pair of nodes is connected with a probability $q$. Thus,

$$p(\textrm{path exists}) = q^{(L-1)}.$$

The probability that there are no other edges from any node (except the last) to any other node besides the next one in the sequence is:

$$p(\textrm{path is replayable }|\textrm{ path exists}) = (1 - q)^{(L-1)(L-2)}.$$

Thus for the ER network we have:

$$E\left[R_L(C_{ER})\right] = \cfrac{N!}{(N-L)!}q^{(L-1)}(1 - q)^{(L-1)(L-2)}.$$

For a fixed $L$, $E[R_L(C_{ER})]$ as a function of $N$ is given by

$$A(L)N(N-1)...(N-L+1) = A(L)(N^L + B(L)N^{L-1} + ...) \sim O(N^L).$$

And so the replay capacity for the ER network is approximately a constant fraction of $N^L$.

We can also show that for $N$ sufficiently large, the expected replay capacity increases with $L$. To do this, we want to show that

$$\cfrac{E[R_{L_2}(C_{ER})]}{E[R_{L_1}(C_{ER})]} > 1$$

when $L_2 > L_1$. This ratio equals

$$\cfrac{\cfrac{N!}{(N-L_2)!}q^{(L_2-1)}(1 - q)^{(L_2-1)(L_2-2)}}{\cfrac{N!}{(N-L_1)!}q^{(L_1-1)}(1 - q)^{(L_1-1)(L_1-2)}}$$

$$ = \cfrac{(N-L_1)!}{(N-L_2)!}q^{L_2-L_1}(1-q)^{(L_2-L_1)(L_1 + L_2 - 3)}$$

$$ = (N-L_1)(N-L_1-1)\dots(N-L_2+1)q^{L_2-L_1}(1-q)^{(L_2-L_1)(L_1 + L_2 - 3)}$$

$$ > (N-L_2+1)^{L_2 - L_2}q^{L_2-L_1}(1-q)^{(L_2-L_1)(L_1 + L_2 - 3)}$$

$$ = (q(N - L_2 + 1))^{L_2 - L_1}(1-q)^{(L_2-L_1)(L_1 + L_2 - 3)} .$$

Since the both terms of this are positive, and the exponent $(L_2 - L_1)$ of the first term is greater than 1, the full quantity will increase monotonically with $N$. Therefore, for $N$ sufficiently large:

$$(q(N - L_2 + 1))^{L_2 - L_1}(1-q)^{(L_2-L_1)(L_1 + L_2 - 3)} > 1.$$

Thus for any $q \notin \{0, 1\}$, the expected replay capacity increases with $L$.

However, since the replay capacity is $0$ when $q = 0$ or $q = 1$, there will be an optimal $q^*$ for each $L$, and this will simply be given by the argmax of 

$$f(q) = q^{L-1}(1 - q)^{(L-1)(L-2)}.$$

We find this by setting to $df/dq = 0$. Specifically:

$$0 = (L-1){q^*}^{L-2}(1 - q^*)^{(L-1)(L-2)} - {q^*}^{L-1}(L-1)(L-2)(1 - q^*)^{(L-1)(L-2) - 1}$$

$$ = (1 - q^*) - q^*(L-2) = 1 - (L - 1)q^*$$

So

$$q^* = \cfrac{1}{L-1}.$$

Thus, the optimal $q$ decreases with $L$, or in other words, increasing the length of the sequences you want to replay requires increasing the sparseness of the network.